In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.inspection import permutation_importance
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

In [27]:
train = pd.read_csv("C:\\Users\\user\\Desktop\\빅데이터 8기 김영규\\ML_DL\\ML_week2\\Day6\\Sproj1\\train.csv")


In [28]:
train

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,traveltime,studytime,...,Fjob_other,Fjob_services,Fjob_teacher,reason_course,reason_home,reason_other,reason_reputation,guardian_father,guardian_mother,guardian_other
0,1,1,16,1,0,1,4,2,1,2,...,1,0,0,1,0,0,0,0,1,0
1,1,1,15,1,1,1,4,4,2,2,...,0,1,0,0,0,0,1,0,1,0
2,1,1,18,1,0,1,3,3,1,2,...,0,0,0,0,1,0,0,1,0,0
3,1,0,16,1,1,1,2,3,1,2,...,0,0,1,0,0,1,0,0,1,0
4,1,1,15,1,1,1,4,3,1,2,...,1,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
312,1,0,16,1,1,1,2,1,1,2,...,1,0,0,1,0,0,0,0,1,0
313,1,0,17,1,1,1,4,3,1,3,...,0,1,0,0,0,0,1,0,1,0
314,1,1,16,1,0,1,2,2,2,2,...,1,0,0,0,0,0,1,0,1,0
315,1,1,18,1,1,1,4,2,1,2,...,1,0,0,0,1,0,0,0,1,0


In [31]:
# RF의 feature name과 feature importance의 DataFrame을 위한 코드입니다.
tinput = train.drop(columns=['G3'])
ttarget = train['G3']
ttrain_input, tval_input, ttrain_target, tval_target = train_test_split(tinput,
                                                                    ttarget,
                                                                    test_size=0.2,
                                                                    random_state=27)

In [30]:
input = train.drop(columns=['G3']).to_numpy()
target = train['G3'].to_numpy()

In [8]:
train_input, val_input, train_target, val_target = train_test_split(input,
                                                                    target,
                                                                    test_size=0.2,
                                                                    random_state=27)

최종 모델
- 중선형 회귀(규제:라쏘)
- 랜덤포레스트(그리드 서치)


In [9]:
# Poly
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(include_bias=False)
poly.fit(train_input)
train_poly = poly.transform(train_input)
val_poly = poly.transform(val_input)

# standard
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
ss.fit(train_poly)

train_scaled = ss.transform(train_poly)
val_scaled = ss.transform(val_poly)

In [10]:
#Lasso
from sklearn.linear_model import Lasso

lasso = Lasso()
lasso.fit(train_scaled, train_target)
print(lasso.score(train_scaled, train_target))
print(lasso.score(val_scaled, val_target))

0.7618325528953864
0.7612531224234598


In [11]:
# Lasso of cross validation
scores = cross_validate(lasso, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8114752503597178 0.7916205883258888


In [10]:
# Random forest(Grid search)
# from sklearn.model_selection import GridSearchCV

#params = {'max_depth': range(1,51,1),
          #'max_features': range(1,51,1),
          #'n_estimators': range(1,51,1) 
          #}

#gs = GridSearchCV(RandomForestRegressor(random_state=27), params, n_jobs=-1)
#gs.fit(train_input, train_target)
#print(gs.best_params_)

In [17]:
# Random forest of cross validation
rf = RandomForestRegressor(max_depth=5, max_features = 35, n_estimators=18, random_state=27)
rf.fit(train_input, train_target)

scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9488020504322867 0.829182570360185


In [18]:
#RF 특성 중요도
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.00000000e+00 1.01661951e-03 8.57302136e-03 2.20853538e-04
 6.76105594e-04 1.36458615e-04 5.22151852e-03 2.75560572e-03
 5.43650231e-03 1.25526519e-02 6.08741918e-03 4.12867125e-03
 2.68475705e-04 5.36172533e-04 4.48004016e-05 5.44502286e-04
 0.00000000e+00 1.60947457e-03 6.33874767e-04 2.18438567e-03
 7.24643295e-03 6.94322764e-03 2.01012833e-04 1.79838708e-03
 4.64562052e-03 8.97094710e-02 8.84104835e-02 7.30806352e-01
 9.35585195e-05 1.77899310e-04 4.22639667e-04 4.50034657e-04
 4.51469404e-04 0.00000000e+00 0.00000000e+00 3.17658231e-03
 5.66350435e-03 4.24002147e-05 1.39923576e-04 3.23298330e-03
 6.36572237e-04 2.37507336e-04 6.41823639e-05 2.82264166e-03
 0.00000000e+00]


In [37]:
# RF의 feature name과 feature importance의 DataFrame
importances = rf.feature_importances_
feature_names = ttrain_input.columns

feature_importances_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
feature_importances_df = feature_importances_df.sort_values(by='Importance', ascending=False)

feature_importances_df

,Feature,Importance
27,G2,0.707100
25,absences,0.105544
26,G1,0.086921
2,age,0.016535
10,failures,0.012282
24,health,0.009339
19,famrel,0.008566
43,guardian_mother,0.008122
21,goout,0.006965
14,activities,0.004083


In [19]:
# 테스트 자료
test_df = pd.read_csv("C:\\Users\\user\\Desktop\\빅데이터 8기 김영규\\ML_DL\\ML_week2\\Day6\\Sproj1\\test.csv")
test_input = test_df.drop(columns=['G3']).to_numpy()
test_target = test_df['G3'].to_numpy()

In [20]:
# 최종 테스트
print(rf.score(test_input, test_target))

0.9099818698666917


In [21]:
# RF 10번 평균
rf = RandomForestRegressor(max_depth=5, max_features = 35, n_estimators=18)
scores = []

for _ in range(10):
    rf.fit(train_input, train_target)
    score = rf.score(test_input, test_target)
    scores.append(score)

final_score = np.mean(scores)
print(final_score)

0.8982449133664321


In [22]:
# Lasso 10번 평균
lasso = Lasso()
Lscores = []

for _ in range(10):
    lasso.fit(train_input, train_target)
    Lscore = lasso.score(test_input, test_target)
    Lscores.append(Lscore)

final_Lscore = np.mean(Lscores)
print(final_Lscore)

0.8549782940740986
